In [133]:
from langchain.agents import load_tools
from langchain.agents.agent_types import AgentType
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import openai
from langchain.tools import DuckDuckGoSearchRun, Tool
load_dotenv(find_dotenv())
search = DuckDuckGoSearchRun()
openai.api_key = os.getenv('OPENAI_API_KEY')
llm = OpenAI(temperature=0)

In [122]:
tools =  load_tools(["wikipedia","llm-math"],
    llm=llm)

tools[0].description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [123]:
agent=initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [125]:
agent.run("what happend to the titan submarine that sank in 2023")



> Entering new  chain...
 I need to find out what happened to the submarine
Action: Wikipedia
Action Input: Titan submarine 2023
Observation: Page: 2023 Titan submersible incident
Summary: On 18 June 2023, Titan, a submersible operated by OceanGate, an American tourism and expeditions company, imploded during its descent in the North Atlantic Ocean, about 320 nautical miles (590 km) off the coast of Newfoundland, Canada. The submersible, carrying five people, was part of an expedition to view the wreck of the Titanic. Communication with Titan was lost 1 hour and 45 minutes into its dive, and authorities were alerted when it failed to resurface at the scheduled time later that day.
After a search lasting nearly 80 hours, a remotely operated underwater vehicle (ROV) discovered a debris field containing parts of Titan, about 500 metres (1,600 ft) from the bow of the Titanic. The search area was informed by the United States Navy’s (USN) sonar detection of an acoustic signature consisten

'On 18 June 2023, Titan, a submersible operated by OceanGate, an American tourism and expeditions company, imploded during its descent in the North Atlantic Ocean, about 320 nautical miles (590 km) off the coast of Newfoundland, Canada. The submersible, carrying five people, was part of an expedition to view the wreck of the Titanic. Communication with Titan was lost 1 hour and 45 minutes into its dive, and authorities were alerted when it failed to resurface at the scheduled time later that day. After a search lasting nearly 80 hours, a remotely operated underwater vehicle (ROV) discovered a debris field containing parts of Titan, about 500 metres (1,600 ft) from the bow of the Titanic. The search and rescue operation was conducted by an international team led by the United States Coast Guard (USCG), USN, and Canadian Coast Guard.'

In [127]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nWikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Wikipedia, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [134]:
search = DuckDuckGoSearchRun()

tools = [ 
    Tool(
        name="Search",
        func=search.run,
        description="Useful for gathing knowledge about subjects you dont know or need to know more about.",
    )
]


template = """
    You are a bro in a discord chat with your best friends.
           the year is 2023.
           your name is gavebot.
           here is some information about you:
                   Your name is: gavebot
                   -Far-Right Politically
                   -Qanon Follower
                   -Alex Jones esque when it comes to conspriacies
                   -Degen Gambler and problem drinker
                   -Short temper and can be an asshole
                   -Loves sports
                   -Extremely edgy and has no filter.
                   -Cant take jokes too well.
                   -Despises Mike Trout
                   -in late 20s but constantly reminiscing about the past glory days of college
                   - love sports
                   - very outspoken is not afraid of any topic will always give their opinion 
    
           you are given the following message: {input}
    
           you are tasked with responding to the message. as relevant information is needed you can use the following tools to get information
           
           {tools}
    
           use the following format:
           Thought: you should ask yourself if you have enough information to respond or if you need to use a tool
           Action: the tool used
           observation: the result of the tool
           (this Thought/Action/Observation should repeat until you have enough information to respond)
           When you feel you have enough information to give a response you should respond how your personality descibed above would respond.
           Thought: i am ready to respond
           Final Answr: the final in character response to the original input message
    
           Begin! Remeber to answer like the character described above would respond.
           {agent_scratchpad}
    """

In [135]:
#Set up a prompt template
from langchain.prompts import StringPromptTemplate
from typing import List

class CustomPromptTemplate(StringPromptTemplate):
    
    template: str
    tools: List[Tool]

    def format(self, *kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\Observation: {observation}\nThought: "
        kwargs['agent_scratchpad'] = thoughts
        kwargs['tools'] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs['tool_name'] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(*kwargs)

NameError: name 'StringPromptTemplate' is not defined